In [ ]:
import os
import matplotlib.pyplot as plt

!pip install nnunetv2 --quiet
!pip install pydicom --quiet
!pip install boxsdk --quiet
!pip install pynrrd --quiet

import nnunetv2
import numpy as np
import pydicom
import zipfile
import re
import gzip
import matplotlib.pyplot as plt
import pandas as pd
import nrrd
import boxsdk
from boxsdk import Client, OAuth2


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import json

In [ ]:
runtime_dir = '/content/runtime_files/'
files_dir = '/content/runtime_files/data_dir'
data_dir = '/content/runtime_files/data_dir/nnUNet_raw'


if not os.path.exists(runtime_dir):
    os.mkdir(runtime_dir)
if not os.path.exists(files_dir):
    os.mkdir(files_dir)
if not os.path.exists(data_dir):
    os.mkdir(data_dir)


In [ ]:
#results_dir =  ''


In [ ]:


import nibabel as nib
from scipy.ndimage import zoom

def resample_mask(mask_nib, new_spacing=(1, 1, 1)):
    """Resamples a nifti mask to a new voxel spacing.

    Args:
        mask_nib: nibabel Nifti1Image object.
        new_spacing: Tuple specifying the desired voxel spacing (mm).

    Returns:
        nibabel Nifti1Image object with the resampled mask.
    """

    # Get original image information
    original_affine = mask_nib.affine
    original_data = mask_nib.get_fdata()
    original_shape = original_data.shape
    original_spacing = nib.affines.voxel_sizes(original_affine)

    # Calculate zoom factors
    zoom_factors = [original_spacing[i] / new_spacing[i] for i in range(3)]

    # Resample the mask
    resampled_data = zoom(original_data, zoom_factors, order=0)  # order=0 for nearest-neighbor interpolation


    # Create new affine transformation
    new_affine = original_affine.copy()
    for i in range(3):
        new_affine[i, i] = new_spacing[i]

    # Create a new nibabel image object
    resampled_mask = nib.Nifti1Image(resampled_data.astype(original_data.dtype), new_affine)


    return resampled_mask



In [ ]:
from scipy.spatial.distance import directed_hausdorff
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Serif"

In [ ]:

dataset = 'Dataset555_OPCONTRAST'
test_predictions = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
print(os.listdir(test_predictions))

gt_dir = '/content/runtime_files/data_dir/nnUNet_raw'
gt_segmentations = os.path.join(gt_dir, dataset, 'labelsTs')
print(os.listdir(gt_segmentations))

predictions = [preds for preds in os.listdir(test_predictions) if preds.endswith('.nii.gz')]
print(predictions)

import matplotlib.pyplot as plt

gtv_preds = []
gtv_gts = []
hd=[]

for prediction in predictions:
    print(prediction)

    mask_pred = nib.load(os.path.join(results_dir, dataset, 'test_predictions_3d_fullres', prediction))
    mask_gt = nib.load(os.path.join(gt_segmentations, prediction))

    # Resample the masks
    resampled_mask_pred = resample_mask(mask_pred)
    resampled_mask_gt = resample_mask(mask_gt)

    print(np.unique(resampled_mask_pred.get_fdata()))
    print(np.unique(resampled_mask_gt.get_fdata()))
    print(np.sum(resampled_mask_pred.get_fdata()))
    print(np.sum(resampled_mask_gt.get_fdata()))
    gtv_preds.append(np.sum(resampled_mask_pred.get_fdata()))
    gtv_gts.append(np.sum(resampled_mask_gt.get_fdata()))

    # Calculate Hausdorff distance for the first pair of masks
    gt_mask = resampled_mask_pred.get_fdata()
    pred_mask = resampled_mask_gt.get_fdata()

    # Find the coordinates of foreground voxels (value 1) in both masks.
    gt_foreground_coords = np.argwhere(gt_mask == 1)
    pred_foreground_coords = np.argwhere(pred_mask == 1)

    if gt_foreground_coords.size == 0 or pred_foreground_coords.size == 0:
        print("One or both foreground masks are empty. Hausdorff distance cannot be computed.")
        hausdorff_distance = np.nan # Handle the empty case
    else:
        hausdorff_distance = max(directed_hausdorff(gt_foreground_coords, pred_foreground_coords)[0],
                                directed_hausdorff(pred_foreground_coords, gt_foreground_coords)[0])

    print(f"Hausdorff Distance: {hausdorff_distance}")
    hd.append(hausdorff_distance)

gtv_gts_cc = [gtv/1000 for gtv in gtv_gts]
gtv_preds_cc = [gtv/1000 for gtv in gtv_preds]

In [ ]:

from scipy.stats import pearsonr
correlation_coefficient, p_value = pearsonr(gtv_gts_cc, gtv_preds_cc)

print(f"Pearson correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")
print("hausdorff distance (mean +/- std dev): ", np.mean(hd), "+/-", np.std(hd))



plt.figure(figsize=(8, 8))
plt.scatter(gtv_gts_cc, gtv_preds_cc)
plt.plot([min(gtv_gts_cc), max(gtv_gts_cc)], [min(gtv_gts_cc), max(gtv_gts_cc)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Ground Truth GTV (mm$^3$)", size=24, )
plt.ylabel("Predicted GTV (mm$^3$)", size=24, )
plt.title("Ground Truth vs. Predicted GTV \n"+"Configuration 1", size = 28)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

# Display correlation coefficient and p-value
textstr = f"Correlation: {correlation_coefficient:.2f}\nP-value: {p_value:.3f}"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=24, verticalalignment='top', bbox=props)

plt.show()




In [ ]:

dataset = 'Dataset556_OPNONCONTRAST'
test_predictions = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
print(os.listdir(test_predictions))

gt_dir = '/content/runtime_files/data_dir/nnUNet_raw'
gt_segmentations = os.path.join(gt_dir, dataset, 'labelsTs')
print(os.listdir(gt_segmentations))

predictions = [preds for preds in os.listdir(test_predictions) if preds.endswith('.nii.gz')]
print(predictions)

import matplotlib.pyplot as plt

gtv_preds = []
gtv_gts = []
hd=[]

for prediction in predictions:
    print(prediction)

    mask_pred = nib.load(os.path.join(results_dir, dataset, 'test_predictions_3d_fullres', prediction))
    mask_gt = nib.load(os.path.join(gt_segmentations, prediction))

    # Resample the masks
    resampled_mask_pred = resample_mask(mask_pred)
    resampled_mask_gt = resample_mask(mask_gt)

    print(np.unique(resampled_mask_pred.get_fdata()))
    print(np.unique(resampled_mask_gt.get_fdata()))
    print(np.sum(resampled_mask_pred.get_fdata()))
    print(np.sum(resampled_mask_gt.get_fdata()))
    gtv_preds.append(np.sum(resampled_mask_pred.get_fdata()))
    gtv_gts.append(np.sum(resampled_mask_gt.get_fdata()))

    # Calculate Hausdorff distance for the first pair of masks
    gt_mask = resampled_mask_pred.get_fdata()
    pred_mask = resampled_mask_gt.get_fdata()

    # Find the coordinates of foreground voxels (value 1) in both masks.
    gt_foreground_coords = np.argwhere(gt_mask == 1)
    pred_foreground_coords = np.argwhere(pred_mask == 1)

    if gt_foreground_coords.size == 0 or pred_foreground_coords.size == 0:
        print("One or both foreground masks are empty. Hausdorff distance cannot be computed.")
        hausdorff_distance = np.nan # Handle the empty case
    else:
        hausdorff_distance = max(directed_hausdorff(gt_foreground_coords, pred_foreground_coords)[0],
                                directed_hausdorff(pred_foreground_coords, gt_foreground_coords)[0])

    print(f"Hausdorff Distance: {hausdorff_distance}")
    hd.append(hausdorff_distance)

gtv_gts_cc = [gtv/1000 for gtv in gtv_gts]
gtv_preds_cc = [gtv/1000 for gtv in gtv_preds]

from scipy.stats import pearsonr
correlation_coefficient, p_value = pearsonr(gtv_gts_cc, gtv_preds_cc)

print(f"Pearson correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")
print("hausdorff distance (mean +/- std dev): ", np.mean(hd), "+/-", np.std(hd))


plt.figure(figsize=(8, 8))
plt.scatter(gtv_gts_cc, gtv_preds_cc)
plt.plot([min(gtv_gts_cc), max(gtv_gts_cc)], [min(gtv_gts_cc), max(gtv_gts_cc)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Ground Truth GTV (mm$^3$)", size=24)
plt.ylabel("Predicted GTV (mm$^3$)", size=24)
plt.title("Ground Truth vs. Predicted GTV \n"+"Configuration 2", size=28)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

# Display correlation coefficient and p-value
textstr = f"Correlation: {correlation_coefficient:.2f}\nP-value: {p_value:.3f}"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=24, verticalalignment='top', bbox=props)

plt.show()



In [ ]:
noncontrast_predictions = [preds.replace("OPTNONCONTRAST", "OPTCONTRASTNONCONTRAST") for preds in predictions]
print(noncontrast_predictions)

In [ ]:

dataset = 'Dataset557_OPCONTRASTNONCONTRAST'
test_predictions = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
print(os.listdir(test_predictions))

gt_dir = '/content/runtime_files/data_dir/nnUNet_raw'
gt_segmentations = os.path.join(gt_dir, dataset, 'labelsTs')
print(os.listdir(gt_segmentations))

predictions = [preds for preds in os.listdir(test_predictions) if preds.endswith('.nii.gz') and preds in noncontrast_predictions]
print(predictions)

import matplotlib.pyplot as plt

gtv_preds = []
gtv_gts = []
hd = []


for prediction in predictions:
    print(prediction)

    mask_pred = nib.load(os.path.join(results_dir, dataset, 'test_predictions_3d_fullres', prediction))
    mask_gt = nib.load(os.path.join(gt_segmentations, prediction))

    # Resample the masks
    resampled_mask_pred = resample_mask(mask_pred)
    resampled_mask_gt = resample_mask(mask_gt)

    print(np.unique(resampled_mask_pred.get_fdata()))
    print(np.unique(resampled_mask_gt.get_fdata()))
    print(np.sum(resampled_mask_pred.get_fdata()))
    print(np.sum(resampled_mask_gt.get_fdata()))
    gtv_preds.append(np.sum(resampled_mask_pred.get_fdata()))
    gtv_gts.append(np.sum(resampled_mask_gt.get_fdata()))


    # Calculate Hausdorff distance for the first pair of masks
    gt_mask = resampled_mask_pred.get_fdata()
    pred_mask = resampled_mask_gt.get_fdata()

    # Find the coordinates of foreground voxels (value 1) in both masks.
    gt_foreground_coords = np.argwhere(gt_mask == 1)
    pred_foreground_coords = np.argwhere(pred_mask == 1)

    if gt_foreground_coords.size == 0 or pred_foreground_coords.size == 0:
        print("One or both foreground masks are empty. Hausdorff distance cannot be computed.")
        hausdorff_distance = np.nan # Handle the empty case
    else:
        hausdorff_distance = max(directed_hausdorff(gt_foreground_coords, pred_foreground_coords)[0],
                                directed_hausdorff(pred_foreground_coords, gt_foreground_coords)[0])

    print(f"Hausdorff Distance: {hausdorff_distance}")
    hd.append(hausdorff_distance)

gtv_gts_cc = [gtv/1000 for gtv in gtv_gts]
gtv_preds_cc = [gtv/1000 for gtv in gtv_preds]

from scipy.stats import pearsonr
correlation_coefficient, p_value = pearsonr(gtv_gts_cc, gtv_preds_cc)

print(f"Pearson correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")
print("hausdorff distance (mean +/- std dev): ", np.mean(hd), "+/-", np.std(hd))



plt.figure(figsize=(8, 8))
plt.scatter(gtv_gts_cc, gtv_preds_cc)
plt.plot([min(gtv_gts_cc), max(gtv_gts_cc)], [min(gtv_gts_cc), max(gtv_gts_cc)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Ground Truth GTV (mm$^3$)", size=24)
plt.ylabel("Predicted GTV (mm$^3$)", size=24)
plt.title("Ground Truth vs. Predicted GTV \n"+"Configuration 3", size=28)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

# Display correlation coefficient and p-value
textstr = f"Correlation: {correlation_coefficient:.2f}\nP-value: {p_value:.3f}"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=24, verticalalignment='top', bbox=props)

plt.show()



In [ ]:

dataset = 'Dataset557_OPCONTRASTNONCONTRAST'
test_predictions = os.path.join(results_dir, dataset, 'test_predictions_3d_fullres')
print(os.listdir(test_predictions))

gt_dir = '/content/runtime_files/data_dir/nnUNet_raw'
gt_segmentations = os.path.join(gt_dir, dataset, 'labelsTs')
print(os.listdir(gt_segmentations))

predictions = [preds for preds in os.listdir(test_predictions) if preds.endswith('.nii.gz') and preds not in noncontrast_predictions]
print(predictions)

import matplotlib.pyplot as plt

gtv_preds = []
gtv_gts = []
hd=[]


for prediction in predictions:
    print(prediction)

    mask_pred = nib.load(os.path.join(results_dir, dataset, 'test_predictions_3d_fullres', prediction))
    mask_gt = nib.load(os.path.join(gt_segmentations, prediction))

    # Resample the masks
    resampled_mask_pred = resample_mask(mask_pred)
    resampled_mask_gt = resample_mask(mask_gt)

    print(np.unique(resampled_mask_pred.get_fdata()))
    print(np.unique(resampled_mask_gt.get_fdata()))
    print(np.sum(resampled_mask_pred.get_fdata()))
    print(np.sum(resampled_mask_gt.get_fdata()))
    gtv_preds.append(np.sum(resampled_mask_pred.get_fdata()))
    gtv_gts.append(np.sum(resampled_mask_gt.get_fdata()))

    # Calculate Hausdorff distance for the first pair of masks
    gt_mask = resampled_mask_pred.get_fdata()
    pred_mask = resampled_mask_gt.get_fdata()

    # Find the coordinates of foreground voxels (value 1) in both masks.
    gt_foreground_coords = np.argwhere(gt_mask == 1)
    pred_foreground_coords = np.argwhere(pred_mask == 1)

    if gt_foreground_coords.size == 0 or pred_foreground_coords.size == 0:
        print("One or both foreground masks are empty. Hausdorff distance cannot be computed.")
        hausdorff_distance = np.nan # Handle the empty case
    else:
        hausdorff_distance = max(directed_hausdorff(gt_foreground_coords, pred_foreground_coords)[0],
                                directed_hausdorff(pred_foreground_coords, gt_foreground_coords)[0])

    print(f"Hausdorff Distance: {hausdorff_distance}")
    hd.append(hausdorff_distance)

gtv_gts_cc = [gtv/1000 for gtv in gtv_gts]
gtv_preds_cc = [gtv/1000 for gtv in gtv_preds]

from scipy.stats import pearsonr
correlation_coefficient, p_value = pearsonr(gtv_gts_cc, gtv_preds_cc)

print(f"Pearson correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")
print("hausdorff distance (mean +/- std dev): ", np.mean(hd), "+/-", np.std(hd))


plt.figure(figsize=(8, 8))
plt.scatter(gtv_gts_cc, gtv_preds_cc)
plt.plot([min(gtv_gts_cc), max(gtv_gts_cc)], [min(gtv_gts_cc), max(gtv_gts_cc)], color='red', linestyle='--')  # Diagonal line
plt.xlabel("Ground Truth GTV (mm$^3$)", size=24)
plt.ylabel("Predicted GTV (mm$^3$)", size=24)
plt.title("Ground Truth vs. Predicted GTV \n"+"Configuration 4", size=28)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

# Display correlation coefficient and p-value
textstr = f"Correlation: {correlation_coefficient:.2f}\nP-value: {p_value:.3f}"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=24, verticalalignment='top', bbox=props)

plt.show()

